In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pickle
import tarfile
import bz2
import sys
from urllib.parse import urlparse

In [2]:
with open('url_id.pickle', 'rb') as file:
    url_id = pickle.load(file)
with open('query_dict/querry_id.pickle', 'rb') as file:
    querry_id = pickle.load(file)
with open('query_dict/enlarged_querrys.pickle', 'rb') as file:
    enlarged_querrys = pickle.load(file)

In [3]:
with open('url_id_std.pickle', 'rb') as file:
    url_id_std = pickle.load(file)
with open('id_url_std.pickle', 'rb') as file:
    id_url_std = pickle.load(file)

In [4]:
def host_cut(ex_str):
    ex_str = ex_str.replace('http://',"")
    ex_str = ex_str.replace('https://',"")
    ex_str = ex_str.replace('www.',"")
    if len(ex_str)>0:
        if ex_str[-1]=='/':
            ex_str=ex_str[:-1]
    ans = ex_str.split('/')
    return ans[0]

In [5]:
host_id = dict()
id_host = dict()

In [6]:
idx1 = -1
for key in tqdm(url_id_std):
    new_key = host_cut(key)
    if new_key not in host_id:
        idx1+=1
        host_id[new_key] = idx1
        id_host[idx1] = new_key

100%|██████████| 582093/582093 [00:02<00:00, 195041.60it/s]


In [7]:
doc_to_host = dict()
for key in tqdm(url_id_std):
    new_key = host_cut(key)
    doc_to_host[url_id_std[key]] = host_id[new_key]

100%|██████████| 582093/582093 [00:03<00:00, 181250.44it/s]


In [8]:
with open('host_id.pickle', 'wb') as file:
    pickle.dump(host_id, file)
with open('id_host.pickle', 'wb') as file:
    pickle.dump(id_host, file)
with open('doc_to_host.pickle', 'wb') as file:
    pickle.dump(doc_to_host, file)

In [9]:
min_timestamp = 1483218001000
raw_data = []
idx = -1
str_index = -1
t = tarfile.open("2017.tar")
for filename in tqdm(t.getnames()):
    #try:
    if filename!='2017' and filename!='2017/_SUCCESS':
        f=t.extractfile(filename)
        source_file = bz2.BZ2File(f, "r")
        idx+=1
        for line in source_file:
            flag = False
            cur_data = []
            a,b,c,d = line.decode('utf-8', errors = 'ignore').strip('\n').split('\t')
            tmp = a.split('@')
            querry = tmp[0]
            if querry in enlarged_querrys:
                cur_data.append(enlarged_querrys[querry])
                flag = True
            else:
                cur_data.append(-1)
            geo = int(tmp[1])
            cur_data.append(geo)
            shown_urls = b.split(',')
            shown_urls = [host_cut(word) for word in shown_urls]
            url_list = []
            loc_bad_dict = dict()
            loc_bad_idx=-1
            for elem in shown_urls:
                if elem in host_id:
                    url_list.append(host_id[elem])
                    flag = True
                else:
                    loc_bad_dict[elem] = loc_bad_idx
                    url_list.append(loc_bad_idx)
                    loc_bad_idx-=1
            cur_data.append(tuple(url_list))
            clicked_urls = c.split(',')
            clicked_urls = [host_cut(word) for word in clicked_urls]
            url_list = []
            for elem in clicked_urls:
                if elem in host_id:
                    url_list.append(host_id[elem])
                else:
                    url_list.append(loc_bad_dict[elem])
            cur_data.append(tuple(url_list))
            timestamps = [int(el) for el in d.split(',')]
            cur_data.append(tuple(timestamps))
            if flag:
                str_index+=1
                cur_data.append(str_index)
                raw_data.append(tuple(cur_data))
        if raw_data==[]:
            print(idx, filename, 'no relevant info')
        with open('rh_{}.pickle'.format(idx), 'wb') as file:
            pickle.dump(tuple(raw_data), file)
        zip_obj= zipfile.ZipFile("rh_{}.zip".format(idx),"w", compression=zipfile.ZIP_DEFLATED)
        zip_obj.write("rh_{}.pickle".format(idx))
        zip_obj.close()
        shutil.copy2("rh_{}.zip".format(idx), 'rh_host_data_upd/'+"rh_{}.zip".format(idx))
        try:
            os.remove("rh_{}.zip".format(idx))
            os.remove("rh_{}.pickle".format(idx))
        except PermissionError:
            sleep(3)
            os.remove("rh_{}.zip".format(idx))
            os.remove("rh_{}.pickle".format(idx))
        raw_data = []
    #except:
        #print(filename)
        #continue
t.close()

 34%|███▍      | 649/1902 [4:57:13<11:29:20, 33.01s/it]

648 2017/part-m-01693.bz2 no relevant info


 38%|███▊      | 718/1902 [5:17:10<3:45:12, 11.41s/it] 

717 2017/part-m-01695.bz2 no relevant info


 43%|████▎     | 818/1902 [5:45:41<4:00:38, 13.32s/it] 

...

 83%|████████▎ | 1578/1902 [9:25:50<1:12:24, 13.41s/it]

1576 2017/part-m-01696.bz2 no relevant info


 90%|████████▉ | 1704/1902 [10:02:39<1:01:03, 18.50s/it]

1702 2017/part-m-01694.bz2 no relevant info


100%|██████████| 1902/1902 [11:05:28<00:00, 20.99s/it]  


In [8]:
url_id

{'smartjs.academy/architecture/': 0,
 'commonappessayhelp.accountant/2016/05/08/essay-on-importance-of-cleanliness-at-home.asp': 1,
 'loading54.supstok.accountant/%D0%94%D0%BE%D1%81%D0%BA%D0%B0+%D0%BD%D0%B0+%D0%B4%D0%B2%D1%83%D1%85+%D0%BA%D0%BE%D0%BB%D0%B5%D1%81%D0%B0%D1%85+%D0%BA%D0%B0%D0%BA+%D0%BD%D0%B0%D0%B7%D1%8B%D0%B2%D0%B0%D0%B5%D1%82%D1%81%D1%8F': 2,
 'www.benenson.ae/viewtopic.php?p=37173&t=12788': 3,
 'kazakhstan.ae/ru/%D0%B0%D0%B1%D1%83-%D0%B4%D0%B0%D0%B1%D0%B8-%D0%BF%D0%BB%D0%B0%D0%B7%D0%B0-%D0%BE%D1%82-25-03-2011%D0%B3/': 4,
 'asus.ucoz.ae/load/programmy/bezopasnost/antisms_8_0_rus/7-1-0-27335': 5,
 'asus.ucoz.ae/load/programmy/ofis/promt_expert_10_build_9_0_526/12-1-0-26633': 6,
 'www.uzbekistanconsulate.ae/25anniversary.php': 7,
 'basel.aero/krasnodar/about-airport/how-to-get/': 8,
 'basel.aero/krasnodar/passengers/transport/': 9,
 'kazan.aero/for-passengers-and-guests/how-to-get/': 10,
 'www.kgd.aero/news/?SHOWALL_1=1': 11,
 'privat.aero/plaines/yakovlev/yak-18': 12,
 'f

In [23]:
host_cut('mandarinka.do.am/forum/7-118-1')

'mandarinka.do.am'

In [18]:
urlparse('novajob.id.ai/mongolskaya-thanka/')

ParseResult(scheme='', netloc='', path='novajob.id.ai/mongolskaya-thanka/', params='', query='', fragment='')